# IAPI Bridge Test

Stand-alone codes using Pythonnet to connect to IAPI

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

from IPython import display
import pylab as plt
import numpy as np
import clr
import atexit

In [3]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')

from vimms.Common import *

## Calling the IAPI Bridge from Python

### Load our custom assembly

In [4]:
ref_dir = 'C:\\Users\\joewa\\Work\\git\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'
if ref_dir not in sys.path:
    sys.path.append(ref_dir)

In [5]:
print(clr.FindAssembly('FusionLibrary'))

C:\Users\joewa\Work\git\vimms\Interface\FusionLibrary\bin\Debug\netstandard2.0\FusionLibrary.dll


In [6]:
list(clr.ListAssemblies(False))

['mscorlib',
 'clrmodule',
 'Python.Runtime',
 'System.Core',
 'System.Configuration',
 'System.Xml',
 'System',
 '__CodeGenerator_Assembly',
 'e__NativeCall_Assembly']

In [7]:
ref = clr.AddReference('FusionLibrary')

In [8]:
list(clr.ListAssemblies(False))

['mscorlib',
 'clrmodule',
 'Python.Runtime',
 'System.Core',
 'System.Configuration',
 'System.Xml',
 'System',
 '__CodeGenerator_Assembly',
 'e__NativeCall_Assembly',
 'FusionLibrary',
 'netstandard',
 'Fusion.API-1.0',
 'API-2.0',
 'Spectrum-1.0']

### Create FusionBridge

Below codes generate a bunch of custom scans

In [9]:
from FusionLibrary import FusionBridge

In [10]:
filename = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
# filename = None # to connect to the actual machine

In [11]:
def scan_handler(scan):
    print('Received scan %s -- %d peaks' % (scan.Header['Scan'], scan.CentroidCount))
    
def state_changed_handler(state):
    print('state_change_handler is called')

class CustomScanHandler(object):
    def __init__(self, fusion_bridge, start_mz, end_mz, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay):
        self.start_mz = start_mz
        self.end_mz = end_mz
        self.isolation_width = isolation_width
        self.collision_energy = collision_energy
        self.ms_level = ms_level
        self.polarity = polarity
        self.first_mass = first_mass
        self.last_mass = last_mass
        self.single_processing_delay = single_processing_delay        
        self.precursor_mass = self.start_mz
        self.fusion_bridge = fusion_bridge
        self.running_number = 100000
            
    def step(self):
        if self.precursor_mass < self.end_mz:
            print('custom_scan_handler called for precursor_mass %f' % self.precursor_mass)
            self.fusion_bridge.CreateCustomScan(self.running_number, self.precursor_mass, self.isolation_width, self.collision_energy, self.ms_level, self.polarity, 
                                                self.first_mass, self.last_mass, self.single_processing_delay)
            self.precursor_mass += 0.02
            self.running_number += 1

In [12]:
start_mz = 524.0
end_mz = 525.0

# custom scan parameters
isolation_width = 0.7
collision_energy = 35.0
ms_level = 2
polarity = POSITIVE
first_mass = 50.0
last_mass = 600.0
single_processing_delay = 0.50

In [13]:
show_console_logs = True
fusion_bridge = FusionBridge(filename, show_console_logs)

In [14]:
show_console_logs = True
fusion_bridge = FusionBridge(filename, show_console_logs)
atexit.register(fusion_bridge.CloseDown)

# create event handlers
scan_handler_delegate = FusionBridge.UserScanArriveDelegate(scan_handler)
state_changed_delegate = FusionBridge.UserStateChangedDelegate(state_changed_handler)
scan_handler = CustomScanHandler(fusion_bridge, start_mz, end_mz, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay)
custom_scan_delegate = FusionBridge.UserCreateCustomScanDelegate(scan_handler.step)

fusion_bridge.SetEventHandlers(scan_handler_delegate, state_changed_delegate, custom_scan_delegate)
fusion_bridge.CreateCustomScan(scan_handler.running_number, scan_handler.precursor_mass, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay)

True

Received scan 2 -- 16 peaks
custom_scan_handler called for precursor_mass 524.000000
Received scan 3 -- 20 peaks
custom_scan_handler called for precursor_mass 524.020000
Received scan 4 -- 22 peaks
custom_scan_handler called for precursor_mass 524.040000
Received scan 5 -- 14 peaks
custom_scan_handler called for precursor_mass 524.060000
Received scan 6 -- 18 peaks
custom_scan_handler called for precursor_mass 524.080000
Received scan 7 -- 33 peaks
custom_scan_handler called for precursor_mass 524.100000
Received scan 8 -- 8 peaks
custom_scan_handler called for precursor_mass 524.120000
Received scan 9 -- 13 peaks
custom_scan_handler called for precursor_mass 524.140000
Received scan 10 -- 11 peaks
custom_scan_handler called for precursor_mass 524.160000
Received scan 12 -- 23 peaks
custom_scan_handler called for precursor_mass 524.180000
Received scan 13 -- 30 peaks
custom_scan_handler called for precursor_mass 524.200000
Received scan 14 -- 5 peaks
custom_scan_handler called for prec

In [15]:
fusion_bridge.CloseDown()